# Simon's Algorithm

In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import numpy as np
import pylatexenc
from sympy import Matrix
import matplotlib.pyplot as plt

In [2]:
s = '10'
n = len(s)

In [3]:
import random
def construct_oracle_map(s):
    seen = set()
    mapping = {}
    for i in range(2**n):
        x = format(i, f'0{n}b')
        x_xor_s = format(i ^ int(s, 2), f'0{n}b')
        if x in seen or x_xor_s in seen:
            continue
        value = format(random.randint(0, 2**n - 1), f'0{n}b')  # 随机 f(x)
        mapping[x] = value
        mapping[x_xor_s] = value
        seen.add(x)
        seen.add(x_xor_s)
    return mapping

In [4]:
oracle_map = construct_oracle_map(s)

In [5]:
def simon_oracle(map_dict):
    qc = QuantumCircuit(n * 2)
    for x, fx in map_dict.items():
        for i, bit in enumerate(x):
            if bit == '1':
                qc.x(i)
        for i, bit in enumerate(fx):
            if bit == '1':
                qc.cx(i, i + n)
        for i, bit in enumerate(x[::-1]):
            if bit == '1':
                qc.x(i)
    return qc

In [6]:
def simon_once(oracle_gate):
    qc = QuantumCircuit(n * 2, n)

    # 初始化输入寄存器为叠加态
    qc.h(range(n))

    # 应用 oracle
    qc.append(oracle_gate.to_gate(), range(n * 2))

    # 对前 n 位再做 Hadamard
    qc.h(range(n))

    # 测量前 n 位
    qc.measure(range(n), range(n))

    sim = AerSimulator(method = 'statevector', device = 'GPU')
    qc_t = transpile(qc, sim)
    result = sim.run(qc_t, shots=1).result()
    counts = result.get_counts()
    return list(counts.keys())[0]

In [7]:
def get_independent_equations(s, oracle_gate, max_trials=20):
    equations = []
    attempts = 0
    while len(equations) < len(s) - 1 and attempts < max_trials:
        y = simon_once(oracle_gate)
        if y == '0'*len(s):
            continue  # 跳过无效结果
        if y not in equations:
            equations.append(y)
        attempts += 1
    return equations


In [8]:
def solve_linear_system(equations):
    mat = []
    for row in equations:
        mat.append([int(b) for b in row])
    M = Matrix(mat)
    null_space = M.nullspace()
    if not null_space:
        return None
    vec = null_space[0] % 2
    return ''.join(str(i) for i in vec)

In [9]:
# 构建 Oracle 电路
oracle_gate = simon_oracle(oracle_map)

# 运行 Simon 算法若干次
eqs = get_independent_equations(s, oracle_gate)


RuntimeError: Simulation device "GPU" is not supported on this system

In [ ]:
# 输出得到的 y·s = 0 的方程组
print("收集到的测量结果（满足 y·s=0）：")
for e in eqs:
    print(e)

# 解出隐藏字符串 s
guessed_s = solve_linear_system(eqs)
print("\n预测出的 s 是：", guessed_s)
print("真实的   s 是：", s)

收集到的测量结果（满足 y·s=0）：

预测出的 s 是： None
真实的   s 是： 1
